 <div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

In [ ]:
import ROOT
from ROOT import TFile, TMVA, TCut

# Import JsMVA and enable JS visualization

In [ ]:
import sys
for path in sys.path:
    if path.find("JsMVA")!=-1:
        sys.path.remove(path)

def delete_module(modname):
    from sys import modules
    try:
        thismod = modules[modname]
    except KeyError:
        raise ValueError(modname)
    these_symbols = dir(thismod)
    del modules[modname]
    for mod in modules.values():
        try:
            delattr(mod, modname)
        except AttributeError:
            pass
delete_module("JsMVA")

In [ ]:
import sys, os
sys.path.append(os.path.expanduser("/home/main/notebooks/src/python/"))
import JsMVA

In [ ]:
%jsmva on

## Declare Factory and DataLoader

In [ ]:
outputFile = TFile("files/TMVAOutput.root", "RECREATE")
inputFile  = TFile("files/mydataset.root")

TMVA.Tools.Instance()

factory = TMVA.Factory("TMVAClassification", outputFile)
loader = TMVA.DataLoader("mydataset")

# General steps
## Adding variables to DataLoader

In [ ]:
# Adding variables to dataset
loader.AddVariable("var0", 'F')
loader.AddVariable("var1", 'F')
loader.AddVariable("var2", 'F')
loader.AddVariable("var3 := var0-var1", 'F')
loader.AddVariable("var4 := var0*var2", 'F')
loader.AddVariable("var5 := var1+var2", 'F')

## Setting up dataset from Trees

In [ ]:
print "--- TMVAClassification       : Using input file:", inputFile.GetName()
   
# Register the training and test trees

tsignal     = inputFile.Get("MyMCSig")
tbackground = inputFile.Get("MyMCBkg")
     
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(tsignal, signalWeight)
loader.AddBackgroundTree(tbackground, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = tsignal
loader.fTreeB = tbackground
loader.PrepareTrainingAndTestTree(mycuts,
                                  mycutb,
                                  "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V")

In [ ]:
factory.BookDNN(loader)

# New method for visualizing input variables

# Input

In [ ]:
loader.DrawInputVariable("var0")

## Transformed input

In [ ]:
loader.DrawInputVariable("var0", processTrfs="D") #I;N;D;P;U;G,D

## Correlation matrix

In [ ]:
loader.DrawCorrelationMatrix("Signal")

# General steps

## Booking classifiers

In [ ]:
# Boosted Decision Trees
factory.BookMethod(loader,
                   TMVA.Types.kBDT,
                   "BDT",
                   "!V:NTrees=5:MinNodeSize=2.5%:MaxDepth=2:CreateMVAPdfs:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20")

# TMVA ANN: MLP Multilayer Perceptrons
factory.BookMethod(loader,
                   TMVA.Types.kMLP,
                   "MLP",
                   "EpochMonitoring:!V:NeuronType=tanh:VarTransform=N:NCycles=150:CreateMVAPdfs:HiddenLayers=N+5:TestRate=5:!UseRegulator")

In [ ]:
layoutString = "Layout=TANH|100,TANH|50,TANH|10,LINEAR"

training0 = "LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,"
training0+= "WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True"
training1 = "LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,"
training1+= "WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1"
training2 = "LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,"
training2+= "WeightDecay=0.0001,Regularization=L2,Multithreading=True"
training3 = "LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,"
training3+= "WeightDecay=0.0001,Regularization=NONE,Multithreading=True"

trainingStrategyString = "TrainingStrategy="
trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3

nnOptions = "!H:V:VarTransform=Normalize:ErrorStrategy=CROSSENTROPY"
#nnOptions = "!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM"
#nnOptions = "!H:V:VarTransform=Normalize:ErrorStrategy=CHECKGRADIENTS"

nnOptions += ":" 
nnOptions += layoutString
nnOptions += ":"
nnOptions += trainingStrategyString

factory.BookMethod(loader, TMVA.Types.kDNN, "DNN", nnOptions )

## Training

In [ ]:
factory.TrainAllMethods()

## Evaluation

In [ ]:
factory.TestAllMethods()

factory.EvaluateAllMethods()

# Classifier output distribution

In [ ]:
factory.DrawOutputDistribution("mydataset", "MLP")

# Classifier Probability Distribution


In [ ]:
factory.DrawProbabilityDistribution("mydataset", "MLP")

# ROC curve

In [ ]:
factory.DrawROCCurve("mydataset")

# Classifier cut efficiencies

In [ ]:
factory.DrawCutEfficiencies("mydataset", "BDT")

# Neural Network

* Mouseover (node, weight): focusing
* Zooming and grab and move supported
* Reset: double click

In [ ]:
factory.DrawNeuralNetwork("mydataset", "MLP")

# Deep Neural network

In [ ]:
factory.DrawNeuralNetwork("mydataset", "DNN")

# Decision Tree

* Mouseover (node, weight): showing decision path
* Zooming and grab and move supported
* Reset: double click
* Click on node: 
    * hiding subtree, if node children are hidden the node will have a green border
    * rescaling: bigger nodes, bigger texts
    * click again to show the subtree

In [ ]:
factory.DrawDecisionTree("mydataset", "BDT")

In [ ]:
outputFile.Close()

In [ ]:
factory.DrawDNNWeights("mydataset", "DNN")